In [87]:
import numpy as np
import pandas as pd
import json
import requests
import bs4
import time
import datetime

# JSON Structure

```
{
    KITAB1:{
        BAB1:{
            AYAT1: 'isi ayat',
            AYAT2: 'isi ayat'
        }
        BAB2:{
            AYAT1: 'isi ayat'
            AYAT2: 'isi ayat'
        }
    }
    KITAB2:{
        BAB1:{
            AYAT1: 'isi ayat',
            AYAT2: 'isi ayat'
        }
        BAB2:{
            AYAT1: 'isi ayat'
            AYAT2: 'isi ayat'
        }
    }
}

```

In [92]:
def kitab_keys():
    URL = 'https://www.sabda.org/alkitab/tb/?kitab=1&pasal=1'
    res = requests.get(URL)
    soup = bs4.BeautifulSoup(res.text)
    
    # Find kitab list
    contents = soup.body.div.find_all(attrs={'class':'wn'})
    
    kitab_keys_crawl = (contents[0].table.find_all('tr'))[2].find_all('a')
    kitab_keys = {'1':'Kejadian'}

    for kitab_key_crawl in kitab_keys_crawl:
        key = (kitab_key_crawl['href']).split('=')
        kitab_keys[key[1]] = kitab_key_crawl.text
        
    return kitab_keys

def number_of_bab(kitab_key):
    URL = f'https://www.sabda.org/alkitab/tb/?kitab={kitab_key}&pasal=1'
    res = requests.get(URL)
    soup = bs4.BeautifulSoup(res.text)
    
    # Find how much "bab" in each kitab
    contents = (soup.body.center.table.tr)
    contents = contents.find_all('td')
    contents = contents[1].find_all('table')
    contents = contents[1].tr.td

    array_of_contents = contents.text.replace('\n', '').split('|')
    number_of_bab = array_of_contents[len(array_of_contents)-1]
    try:
        return int(number_of_bab)
    except ValueError:
        return 1

def ayat(kitab_key, bab_number):
    URL = f'https://www.sabda.org/alkitab/tb/?kitab={kitab_key}&pasal={bab_number}'
    res = requests.get(URL)
    soup = bs4.BeautifulSoup(res.text)
    
    # Fetch ayat-ayat in bab
    contents = (soup.body.center.table.tr)
    contents = contents.find_all('td')
    contents = contents[1].find_all('table')
    contents = contents[2]
    contents = contents.find_all(attrs={'width':'92%'})
    ayat_ayat = []
    for content in contents:
        ayat_ayat.append(content.text)
    return ayat_ayat



In [93]:
RESULTS_FILE = './alkitab_data.json'
PROGRESS_CACHE = './progress_check.txt'

try:
    with open(RESULTS_FILE, 'r') as f:
        RESULTS_DICT = json.load(f)
    with open(PROGRESS_CACHE, 'r') as f:
        HOTSTART = int(f.read())
except FileNotFoundError:
    RESULTS_DICT = {}
    HOTSTART = 0

loop = 0
KITAB_KEYS = kitab_keys()
# Loop for every kitab
for KITAB_KEY in KITAB_KEYS:
    tick = datetime.datetime.now()
    n_bab = number_of_bab(KITAB_KEY)
    
    if not (loop < HOTSTART):
        RESULTS_DICT[KITAB_KEYS[KITAB_KEY]] = {}
    
    for bab in range(n_bab):
        
        if loop < HOTSTART:
            loop+=1
            continue
            
        list_of_ayat_ayat = ayat(KITAB_KEY, bab)
        RESULTS_DICT[KITAB_KEYS[KITAB_KEY]][bab] = list_of_ayat_ayat
        
        loop+=1
        
        # Save progress to file
        with open(RESULTS_FILE, 'w') as f:
            json.dump(RESULTS_DICT, f)
        with open(PROGRESS_CACHE, 'w') as f:
            f.write(str(loop))
            
        time.sleep(2)
        
        
    tock = datetime.datetime.now()
        
    print(f'sabda.org {loop}/{len(KITAB_KEYS)} : {KITAB_KEYS[KITAB_KEY]} Finished. Time elapsed {tock-tick}')
        

sabda.org 50/66 : Kejadian Finished. Time elapsed 0:00:00.059000
sabda.org 90/66 : Keluaran Finished. Time elapsed 0:00:00.117663
sabda.org 117/66 : Imamat Finished. Time elapsed 0:00:00.097759
sabda.org 153/66 : Bilangan Finished. Time elapsed 0:00:00.106217
sabda.org 187/66 : Ulangan Finished. Time elapsed 0:00:00.103608
sabda.org 211/66 : Yosua Finished. Time elapsed 0:00:00.089612
sabda.org 232/66 : Hakim-hakim Finished. Time elapsed 0:00:00.103421
sabda.org 236/66 : Rut Finished. Time elapsed 0:00:00.095964
sabda.org 267/66 : 1 Samuel Finished. Time elapsed 0:00:00.101957
sabda.org 291/66 : 2 Samuel Finished. Time elapsed 0:00:00.099238
sabda.org 313/66 : 1 Raja-raja Finished. Time elapsed 0:00:00.104251
sabda.org 338/66 : 2 Raja-raja Finished. Time elapsed 0:00:00.102623
sabda.org 367/66 : 1 Tawarikh Finished. Time elapsed 0:00:00.097901
sabda.org 403/66 : 2 Tawarikh Finished. Time elapsed 0:00:00.093969
sabda.org 413/66 : Ezra Finished. Time elapsed 0:00:00.097138
sabda.org 426/

In [86]:
a = {'a':{'c':'adaf', 'd':'sdgdf'}, 'b':{}}
a['a'] = {}
a

{'a': {}, 'b': {}}

In [78]:
# Find how much "bab" in each kitab
contents = (soup.body.center.table.tr)
contents = contents.find_all('td')
contents = contents[1].find_all('table')
contents = contents[2]
contents = contents.find_all(attrs={'width':'92%'})
ayat_ayat = []
for content in contents:
    ayat_ayat.append(content.text)
    
ayat_ayat

['Pada mulanya Allah menciptakan langit dan bumi.',
 'Bumi belum berbentuk dan kosong; gelap gulita menutupi samudera raya, dan Roh Allah melayang-layang di atas permukaan air.',
 'Berfirmanlah Allah: "Jadilah terang." Lalu terang itu jadi.',
 'Allah melihat bahwa terang itu baik, lalu dipisahkan-Nyalah terang itu dari gelap.',
 'Dan Allah menamai terang itu siang, dan gelap itu malam. Jadilah petang dan jadilah pagi, itulah hari pertama.',
 'Berfirmanlah Allah: "Jadilah cakrawala di tengah segala air untuk memisahkan air dari air."',
 'Maka Allah menjadikan cakrawala dan Ia memisahkan air yang ada di bawah cakrawala itu dari air yang ada di atasnya. Dan jadilah demikian.',
 'Lalu Allah menamai cakrawala itu langit. Jadilah petang dan jadilah pagi, itulah hari kedua.',
 'Berfirmanlah Allah: "Hendaklah segala air yang di bawah langit berkumpul pada satu tempat, sehingga kelihatan yang kering." Dan jadilah demikian.',
 'Lalu Allah menamai yang kering itu darat, dan kumpulan air itu dinam

## kitab_keys_crawl = (contents[0].table.find_all('tr'))[2].find_all('a')
kitab_keys = {'1':'Kejadian'}

for kitab_key_crawl in kitab_keys_crawl:
    key = (kitab_key_crawl['href']).split('=')
    kitab_keys[key[1]] = kitab_key_crawl.text
kitab_keys

In [26]:
print(contents[0].table.prettify())

<table border="0" cellpadding="0" cellspacing="0" width="100%">
 <tr>
  <td class="wn" colspan="7" valign="top">
   Mini SABDAweb adalah versi sederhana dari
   <a href="http://sabdaweb.sabda.org/">
    SABDAweb
   </a>
   .
   <br/>
   Untuk Alkitab online yang lebih lengkap dengan aneka versi Alkitab 
	    	yang dapat dibaca secara paralel serta dukungan
	    	alat-alat Biblika (kamus, konkordansi, leksikon, peta),
	    	silakan kunjungi
   <a href="http://sabdaweb.sabda.org/">
    situs SABDAweb
   </a>
   .
   <br/>
   Untuk SABDAweb versi terbaru, yang bernama Alkitab SABDA, yang berisi bahan-bahan yang lebih lengkap lagi, disertai versi Alkitab yang jauh lebih banyak, silakan kunjungi
   <a href="http://alkitab.sabda.org">
    situs Alkitab SABDA
   </a>
   .
  </td>
 </tr>
 <tr>
  <td class="wn1" colspan="4" valign="center">
   <img border="0" src="/_images/pub/pnt_part_1.gif"/>
   <b>
    Perjanjian Lama
   </b>
  </td>
  <td class="wn1" colspan="3" height="30" valign="center">